# Creating required submission and anomaly files

In [ ]:
import os, sys, numpy as np, pandas as pd

# ---- Config (matches challenge spec) ----
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
SUB_PATH  = "submission.csv"
ANOM_PATH = "anomalies.csv"
REQUIRED_SUB_COLS  = ["Date", "Predicted_Total_Calls"]
REQUIRED_ANOM_COLS = ["Date","Actual","Expected","Anomaly_Score","Note"]

def _assert_columns(df, cols, name):
    if list(df.columns) != cols:
        raise AssertionError(f"{name} columns must be {cols} (got {list(df.columns)})")

def _assert_date_format(series, name):
    try:
        pd.to_datetime(series, format="%Y-%m-%d", errors="raise")
    except Exception as e:
        raise AssertionError(f"{name} has invalid date format (expect YYYY-MM-DD): {e}")

# optional local train (not required for reproducibility)
train = None
if os.path.exists("data/train.csv"):
    train = pd.read_csv("data/train.csv", parse_dates=["date"])
    if "total_calls" not in train.columns:
        raise AssertionError("data/train.csv must include columns: date,total_calls")

# Use fixed scoring window per brief
START, END = "2025-05-01", "2025-08-01"
future_dates = pd.date_range(START, END, freq="D")

# Constant forecast (deterministic); replaced later by the real model
const_val = float(train["total_calls"].iloc[-1]) if train is not None else 10000.0

submission = pd.DataFrame({
    "Date": future_dates.strftime("%Y-%m-%d"),
    "Predicted_Total_Calls": const_val
})[REQUIRED_SUB_COLS]
_assert_columns(submission, REQUIRED_SUB_COLS, "submission.csv")
_assert_date_format(submission["Date"], "submission.Date")
submission.to_csv(SUB_PATH, index=False)

# anomalies placeholder with correct header
anomalies = pd.DataFrame(columns=REQUIRED_ANOM_COLS)
_assert_columns(anomalies, REQUIRED_ANOM_COLS, "anomalies.csv")
anomalies.to_csv(ANOM_PATH, index=False)

print(f"✅ Wrote {SUB_PATH} ({submission.shape}) and {ANOM_PATH} ({anomalies.shape}). All checks passed.")


✅ Wrote submission.csv ((93, 2)) and anomalies.csv ((0, 5)). All checks passed.


# Fetch & aggregate NYC 311 data into daily train/test CSVs

In [ ]:
# Windows per challenge (training/features) and (forecast/scoring)
TRAIN_START, TRAIN_END = "2024-08-01", "2025-04-30"
TEST_START,  TEST_END  = "2025-05-01", "2025-08-01"

import time, requests, pandas as pd, numpy as np, os
os.makedirs("data", exist_ok=True)

BASE = "https://data.cityofnewyork.us/resource/erm2-nwe9.json"

def fetch_daily_counts(start_ymd: str, end_ymd: str) -> pd.DataFrame:
    """
    Robust SoQL aggregation by calendar date with retries & paging.
    Returns columns: date,total_calls,manhattan,bronx,brooklyn,queens,staten_island
    """
    select = (
        "date_trunc_ymd(created_date) as date,"
        "count(1) as total_calls,"
        "sum(case when borough='MANHATTAN' then 1 else 0 end) as manhattan,"
        "sum(case when borough='BRONX' then 1 else 0 end) as bronx,"
        "sum(case when borough='BROOKLYN' then 1 else 0 end) as brooklyn,"
        "sum(case when borough='QUEENS' then 1 else 0 end) as queens,"
        "sum(case when borough='STATEN ISLAND' then 1 else 0 end) as staten_island"
    )
    limit, offset, frames = 50000, 0, []
    while True:
        params = {
            "$select": select,
            "$where": f"created_date between '{start_ymd}T00:00:00' and '{end_ymd}T23:59:59'",
            "$group": "date_trunc_ymd(created_date)",
            "$order": "date_trunc_ymd(created_date)",
            "$limit": limit, "$offset": offset
        }
        for attempt in range(3):
            try:
                r = requests.get(BASE, params=params, timeout=120)
                r.raise_for_status()
                chunk = pd.DataFrame(r.json())
                break
            except Exception as e:
                if attempt == 2:
                    raise
                print(f"⚠️ Retry {attempt+1}/3 offset={offset} due to: {e}")
                time.sleep(5)
        if chunk.empty: break
        frames.append(chunk)
        if len(chunk) < limit: break
        offset += limit

    if not frames:
        raise RuntimeError("API returned no rows. Check dates/network.")
    df = pd.concat(frames, ignore_index=True)
    df["date"] = pd.to_datetime(df["date"]).dt.date
    for c in ["total_calls","manhattan","bronx","brooklyn","queens","staten_island"]:
        df[c] = pd.to_numeric(df[c], errors="coerce").fillna(0).astype(int)
    df = df.sort_values("date").reset_index(drop=True)

    # contiguity check (no missing calendar days)
    full = pd.date_range(start_ymd, end_ymd, freq="D").date
    have = df["date"].tolist()
    missing = sorted(set(full) - set(have))
    if missing:
        raise AssertionError(f"Missing days: {missing[:5]} ... total {len(missing)}")
    return df

train_df = fetch_daily_counts(TRAIN_START, TRAIN_END)
test_df  = fetch_daily_counts(TEST_START,  TEST_END)

train_df.to_csv("data/train.csv", index=False)
test_df.to_csv("data/test.csv", index=False)

print("✅ Saved data/train.csv and data/test.csv")
print("Train:", train_df["date"].min(), "→", train_df["date"].max(), "| rows:", len(train_df))
print("Test :", test_df["date"].min(),  "→", test_df["date"].max(),  "| rows:", len(test_df))


✅ Saved data/train.csv and data/test.csv
Train: 2024-08-01 → 2025-04-30 | rows: 273
Test : 2025-05-01 → 2025-08-01 | rows: 93


#  Leak-safe features, train XGBoost, predict scoring window

In [25]:
import pandas as pd, numpy as np, os, xgboost as xgb

# ---- windows (must match brief) ----
TRAIN_START, TRAIN_END = "2024-08-01", "2025-04-30"
TEST_START,  TEST_END  = "2025-05-01", "2025-08-01"

# ---- load daily aggregates built in Step 5 (if missing, raise) ----
assert os.path.exists("data/train.csv"), "Missing data/train.csv. Run Step 5 first."
train_df = pd.read_csv("data/train.csv", parse_dates=["date"])
print("Train rows:", len(train_df), train_df["date"].min().date(), "→", train_df["date"].max().date())

# ---- Feature builder (robust to Series or DatetimeIndex) ----
def make_calendar_features(dates) -> pd.DataFrame:
    """
    Accepts pd.Series, list-like, or DatetimeIndex.
    Returns a DataFrame with calendar & cyclic features (no target lags).
    """
    dt = pd.to_datetime(dates)
    # Ensure we have a Series so .dt works even for DatetimeIndex
    if isinstance(dt, pd.DatetimeIndex):
        dt = pd.Series(dt)
    df = pd.DataFrame({"date": dt.dt.date})
    df["dow"] = dt.dt.dayofweek                  # 0-6
    df["is_weekend"] = (df["dow"] >= 5).astype(int)
    df["dayofyear"] = dt.dt.dayofyear
    # .isocalendar() returns a frame with 'week' (pandas>=1.1)
    df["weekofyear"] = dt.dt.isocalendar().week.astype(int)
    df["month"] = dt.dt.month
    # cyclic encodings
    df["doy_sin"] = np.sin(2*np.pi*df["dayofyear"]/366)
    df["doy_cos"] = np.cos(2*np.pi*df["dayofyear"]/366)
    df["mon_sin"] = np.sin(2*np.pi*df["month"]/12)
    df["mon_cos"] = np.cos(2*np.pi*df["month"]/12)
    df["woy_sin"] = np.sin(2*np.pi*df["weekofyear"]/53)
    df["woy_cos"] = np.cos(2*np.pi*df["weekofyear"]/53)
    # linear trend from train start
    df["t"] = (pd.to_datetime(df["date"]) - pd.to_datetime(TRAIN_START)).dt.days
    return df

# ---- build training matrix ----
train_feat = make_calendar_features(train_df["date"])
train_feat["y"] = train_df["total_calls"].astype(float)

FEATURES = [c for c in train_feat.columns if c not in ["date","y"]]
Xtr, ytr = train_feat[FEATURES], train_feat["y"]

# ---- quick internal validation (last 28 days) ----
split_idx = len(train_feat) - 28
X_fit, y_fit = Xtr.iloc[:split_idx], ytr.iloc[:split_idx]
X_val, y_val = Xtr.iloc[split_idx:], ytr.iloc[split_idx:]

model = xgb.XGBRegressor(
    n_estimators=400, max_depth=6, learning_rate=0.05,
    subsample=0.9, colsample_bytree=0.9, random_state=42, n_jobs=0
)
model.fit(X_fit, y_fit)
val_pred = model.predict(X_val)
rmse = np.sqrt(np.mean((val_pred - y_val.values)**2))
mape = np.mean(np.abs((y_val.values - val_pred) / np.clip(y_val.values, 1e-6, None))) * 100
print(f"Val RMSE≈{rmse:,.0f} | MAPE≈{mape:.2f}% on last 28d")

# ---- refit on all train data ----
model.fit(Xtr, ytr)

# ---- build scoring features (dates only; no leakage) ----
future_dates = pd.date_range(TEST_START, TEST_END, freq="D")
Xte = make_calendar_features(future_dates)

# ---- weekday baseline (strong simple prior) ----
dow_mean = train_feat.groupby("dow")["y"].mean()
pred_dow = Xte["dow"].map(dow_mean).values

# ---- ML prediction + robust blend ----
pred_ml = model.predict(Xte[[c for c in FEATURES]])
pred = 0.7*pred_ml + 0.3*pred_dow

# ---- write submission.csv (exact schema) ----
submission = pd.DataFrame({
    "Date": future_dates.strftime("%Y-%m-%d"),
    "Predicted_Total_Calls": pred
})[["Date","Predicted_Total_Calls"]]
submission.to_csv("submission.csv", index=False)
print("✅ submission.csv written:", submission.shape)


Train rows: 273 2024-08-01 → 2025-04-30
Val RMSE≈553 | MAPE≈4.93% on last 28d
✅ submission.csv written: (93, 2)


# Compute anomalies

In [26]:
import pandas as pd, os, numpy as np

ANOM_COLS = ["Date","Actual","Expected","Anomaly_Score","Note"]
dates = pd.read_csv("submission.csv")["Date"]
expected = pd.read_csv("submission.csv")["Predicted_Total_Calls"]

if os.path.exists("data/test.csv"):
    test_df = pd.read_csv("data/test.csv", parse_dates=["date"])
    if "total_calls" in test_df.columns:
        actual_map = test_df.set_index(test_df["date"].dt.strftime("%Y-%m-%d"))["total_calls"]
        actual = dates.map(actual_map).astype("float")
        df = pd.DataFrame({
            "Date": dates,
            "Actual": actual.round().astype("Int64"),
            "Expected": expected.astype(float)
        })
        df["Anomaly_Score"] = (df["Actual"] - df["Expected"]) / df["Expected"].clip(lower=1e-6)
        df["Note"] = ""
        # select top-5 spikes & top-5 dips for readability (still fine to output all)
        spikes = df.nlargest(5, "Anomaly_Score")
        dips   = df.nsmallest(5, "Anomaly_Score")
        out = pd.concat([spikes, dips]).sort_values("Date")
        out = out[ANOM_COLS]
        out.to_csv("anomalies.csv", index=False)
        print("✅ anomalies.csv written with top-5 spikes & dips:", out.shape)
    else:
        pd.DataFrame(columns=ANOM_COLS).to_csv("anomalies.csv", index=False)
        print("ℹ️ data/test.csv has no 'total_calls'; wrote empty anomalies.csv")
else:
    pd.DataFrame(columns=ANOM_COLS).to_csv("anomalies.csv", index=False)
    print("ℹ️ data/test.csv not found; wrote empty anomalies.csv")


✅ anomalies.csv written with top-5 spikes & dips: (10, 5)
